<h1 style="text-align: center; color: RED">FEATURE EXTRACTION(ÖZELLİK ÇIKARIMI)</h1>

Ham veriden değişken türetmeye özellik çıkarımı denir. Yapısal olabilecek değişkenlerden özellik çıkarma demek var olan bazı değişkenler üzerinden yeni değişkenler türetmek demektir.

![Test](datasets/13.png)

Yapısal olmayan(bilgisayarın anlamadığı veri)veriden özellik çıkarımı:

![Test](datasets/14.png)
![Test](datasets/15.png)

<span style="color: blue; font-weight: bold;">BINARY FEATURES: FLAG,BOOL,TRUE-FALSE</span>

In [10]:
import numpy as np
import pandas as pd
import seaborn as sns
from matplotlib import pyplot as plt
import missingno as msno #eksik değerler için kullanılan bir kütüphane
from datetime import date
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split
from sklearn.neighbors import LocalOutlierFactor
from sklearn.preprocessing import MinMaxScaler, LabelEncoder, StandardScaler, RobustScaler

#Görsel ayaralama için
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
pd.set_option('display.float_format', lambda x: '%.3f' % x)
pd.set_option('display.width', 500)

In [12]:
def load():
    data = pd.read_csv("C:\\Users\\ASLI\\Desktop\\Data Scientist Bootcamp\\6.Feature Engineering\\datasets\\titanic.csv")
    return data

In [54]:
df = load()
df.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.000,1,0,A/5 21171,7.250,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.000,1,0,PC 17599,71.283,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.000,0,0,STON/O2. 3101282,7.925,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.000,1,0,113803,53.100,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.000,0,0,373450,8.050,NaN,S


In [22]:
#Amacım kabin değişkenindeki boş Nan değerlere 0 dolu değerlere 1 değeri vermek.
df['NEW_CABIN_BOOL'] = df['Cabin'].notnull().astype('int')

In [24]:
df.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked,NEW_CABIN_BOOL
0,1,0,3,"Braund, Mr. Owen Harris",male,22.000,1,0,A/5 21171,7.250,NaN,S,0
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.000,1,0,PC 17599,71.283,C85,C,1
2,3,1,3,"Heikkinen, Miss. Laina",female,26.000,0,0,STON/O2. 3101282,7.925,NaN,S,0
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.000,1,0,113803,53.100,C123,S,1
4,5,0,3,"Allen, Mr. William Henry",male,35.000,0,0,373450,8.050,NaN,S,0


In [26]:
#Bağımlı değişken ile yeni oluşturduğumuz değişkeni analiz ediyoruz.
#Sonuca baktığımızda kabin numarası olanların hayatta kalma oranın daha yüksek olduğunu görüyoruz. 
#Bu değişkeni türetmeden önce cabin değişkeni bizim için anlam ifade etmiyordu
df.groupby('NEW_CABIN_BOOL').agg({'Survived':'mean'})

,Survived
NEW_CABIN_BOOL,
0,0.300
1,0.667


* Yukarıdaki oranın rastlantısal olarak mı olup olmadığını kontrol etmek için 2 grup oran testi yapıyoruz.

In [29]:
from statsmodels.stats.proportion import proportions_ztest

In [31]:
test_stat, pvalue = proportions_ztest(count=[df.loc[df["NEW_CABIN_BOOL"] == 1, "Survived"].sum(),
                                             df.loc[df["NEW_CABIN_BOOL"] == 0, "Survived"].sum()],

                                      nobs=[df.loc[df["NEW_CABIN_BOOL"] == 1, "Survived"].shape[0],
                                            df.loc[df["NEW_CABIN_BOOL"] == 0, "Survived"].shape[0]])

In [33]:
print('Test Stat = %.4f, p-value = %.4f' % (test_stat, pvalue))

Test Stat = 9.4597, p-value = 0.0000


* H0= iki grubun oranları arasında fark yoktur.
* H1= fark vardır 
* p-value = 0.0000 < 0.05 oldugu için H0 hipotezi rededdilir yani oranla arasında bir fark vardır gibi görünüyor.
* Fakat bu genellemeyi yapabilmek için çok değişkenli etkiyi de gözlemlememiz gerekiyor.

In [37]:
#Başka değişken üzerinden 1-0 türetmek. Gemiye binen kişinin akrabası var mı yoksa yanlız mı?
df.loc[(df['SibSp'] + df['Parch'])>0,'NEW_IS_ALONE'] = 'NO'
df.loc[(df['SibSp'] + df['Parch'])==0,'NEW_IS_ALONE'] = 'YES'

In [41]:
#Bağımlı değişken ile analizi
df.groupby('NEW_IS_ALONE').agg({'Survived':'mean'})

,Survived
NEW_IS_ALONE,
NO,0.506
YES,0.304


In [43]:
#İstatistiki olarak bir fark varmı ?
test_stat, pvalue = proportions_ztest(count=[df.loc[df["NEW_IS_ALONE"] == 'YES', "Survived"].sum(),
                                             df.loc[df["NEW_IS_ALONE"] == 'NO', "Survived"].sum()],

                                      nobs=[df.loc[df["NEW_IS_ALONE"] == 'YES', "Survived"].shape[0],
                                            df.loc[df["NEW_IS_ALONE"] == 'NO', "Survived"].shape[0]])

In [45]:
print('Test Stat = %.4f, p-value = %.4f' % (test_stat, pvalue))

Test Stat = -6.0704, p-value = 0.0000


* H0= iki grubun oranları arasında fark yoktur.
* H1= fark vardır 
* p-value = 0.0000 < 0.05 oldugu için H0 hipotezi rededdilir yani oranla arasında bir fark vardır gibi görünüyor.
* Fakat bu genellemeyi yapabilmek için çok değişkenli etkiyi de gözlemlememiz gerekiyor. BU değişkenin bir etkisinin olup olmadığını tam olarak modelleme bölümünde görebiliriz.

<span style="color: blue; font-weight: bold;">TEXT'LER ÜZERİNDEN ÖZELLİK ÜRETMEK</span>

In [56]:
df.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.000,1,0,A/5 21171,7.250,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.000,1,0,PC 17599,71.283,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.000,0,0,STON/O2. 3101282,7.925,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.000,1,0,113803,53.100,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.000,0,0,373450,8.050,NaN,S


In [ ]:
#1-LETTER COUNT(İsimlerdeki tüm harfleri saymak)
df['NEW_NAME_COUNT'] = df['Name'].str.len()# tüm harfleri sayacak

In [ ]:
#Kelimleri saydırma
df['NEW_NAME_WORD_COUNT'] = df['Name'].apply(lambda x:len(str(x).split(' ')))

In [ ]:
#Özel yapıları yakalamak.Örneğin doktor olanları ayırt etme
df['NEW_NAME_DR'] = df['Name'].apply(lambda x :len([for x in x.split if x.startswith('Dr')]))

<span style="color: blue; font-weight: bold;">REGEX FEATURES</span>

Bu kısımda bir değişken türetmek için regex için bir örüntü elde etmem.z gerekir. Örneğin Mr. ve Miss. yazımından önce boşuluk var ,ilk harf büyük sonrası küçük ve hep nokta ile bitmiş.

In [62]:
df['NEW_TITLE'] = df.Name.str.extract(r' ([A-Za-z]+)\.', expand=False)

* df.Name df DataFrame'indeki "Name" sütununu temsil eder.
* .str: Pandas'taki bir string (metin) işlemleri için kullanılan özel bir nesnedir. Bu, tüm Name sütununa string (metin) metodlarını uygulamamızı sağlar.
* extract(): Düzenli ifadeler (regex) kullanarak belirli bir deseni bulup çıkarır.
* Eğer expand=True olsaydı, extract() sonucu bir DataFrame olarak döndürürdü.
* expand=False kullanımı, verileri tek bir pandas Serisi (Series) olarak döndürmesini sağlar.
* r' ([A-Za-z]+)\.' → Regex deseni:

| Kısım             | Anlamı                                                                                                                                                                                                  |
| ----------------- | ------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------- |
| **` `** (boşluk)  | Eşleşmeden önce bir boşluk olmalı. Bu, genellikle ismin öncesinde bir boşluk olduğu varsayımı.                                                                                                          |
| **`([A-Za-z]+)`** | Parantez içindeki ifade **yakalanacak grup**. <br> `A-Z` → büyük harfler <br>`a-z` → küçük harfler <br>`+` → bir veya daha fazla harf. <br> Yani **“Mr”, “Mrs”, “Dr”, “Miss” gibi kelimeleri yakalar**. |
| **`\.`**          | Nokta karakterini temsil eder (regex’te `.` normalde “herhangi bir karakter” demektir; `\.` ile gerçek nokta anlamına gelir).                                                                           |


In [66]:
df.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked,NEW_TITLE
0,1,0,3,"Braund, Mr. Owen Harris",male,22.000,1,0,A/5 21171,7.250,NaN,S,Mr
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.000,1,0,PC 17599,71.283,C85,C,Mrs
2,3,1,3,"Heikkinen, Miss. Laina",female,26.000,0,0,STON/O2. 3101282,7.925,NaN,S,Miss
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.000,1,0,113803,53.100,C123,S,Mrs
4,5,0,3,"Allen, Mr. William Henry",male,35.000,0,0,373450,8.050,NaN,S,Mr


In [68]:
#Elde ettiğimiz değişkeni hayatta kalma ve yaş acısından analiz ediyoruz.
#Daha önce yaşın NAN değerlerini cinsiyet kırılımında doldurmuştuk şimdi ise ünvanb bazlı kırlım yapıp doldurabilirz. 
df[["NEW_TITLE", "Survived", "Age"]].groupby(["NEW_TITLE"]).agg({"Survived": "mean", "Age": ["count", "mean"]})

Survived   Age       
              mean count   mean
NEW_TITLE                      
Capt         0.000     1 70.000
Col          0.500     2 58.000
Countess     1.000     1 33.000
Don          0.000     1 40.000
Dr           0.429     6 42.000
Jonkheer     0.000     1 38.000
Lady         1.000     1 48.000
Major        0.500     2 48.500
Master       0.575    36  4.574
Miss         0.698   146 21.774
Mlle         1.000     2 24.000
Mme          1.000     1 24.000
Mr           0.157   398 32.368
Mrs          0.792   108 35.898
Ms           1.000     1 28.000
Rev          0.000     6 43.167
Sir          1.000     1 49.000

<span style="color: blue; font-weight: bold;">DATE FEATURES</span>

In [72]:
dff = pd.read_csv("C:\\Users\\ASLI\\Desktop\\Data Scientist Bootcamp\\6.Feature Engineering\\datasets\\course_reviews.csv")

In [82]:
dff.head()

,Rating,Timestamp,Enrolled,Progress,Questions Asked,Questions Answered
0,5.000,2021-02-05 07:45:55,2021-01-25 15:12:08,5.000,0.000,0.000
1,5.000,2021-02-04 21:05:32,2021-02-04 20:43:40,1.000,0.000,0.000
2,4.500,2021-02-04 20:34:03,2019-07-04 23:23:27,1.000,0.000,0.000
3,5.000,2021-02-04 16:56:28,2021-02-04 14:41:29,10.000,0.000,0.000
4,4.000,2021-02-04 15:00:24,2020-10-13 03:10:07,10.000,0.000,0.000


In [80]:
#Amacımız timestamp üzerinden yeni değişkenler üretmek
#İLk olarak bu değişkenin veri tipini zaman olarak değiştirelim
dff['Timestamp'] = pd.to_datetime(dff['Timestamp'], format='%Y-%m-%d %H:%M:%S')

In [84]:
dff.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4323 entries, 0 to 4322
Data columns (total 6 columns):
 #   Column              Non-Null Count  Dtype         
---  ------              --------------  -----         
 0   Rating              4323 non-null   float64       
 1   Timestamp           4323 non-null   datetime64[ns]
 2   Enrolled            4323 non-null   object        
 3   Progress            4323 non-null   float64       
 4   Questions Asked     4323 non-null   float64       
 5   Questions Answered  4323 non-null   float64       
dtypes: datetime64[ns](1), float64(4), object(1)
memory usage: 202.8+ KB


In [86]:
#Yıl türetme
dff['year'] = dff['Timestamp'].dt.year

In [88]:
dff['month'] = dff['Timestamp'].dt.month

In [90]:
dff.head()

,Rating,Timestamp,Enrolled,Progress,Questions Asked,Questions Answered,year,month
0,5.000,2021-02-05 07:45:55,2021-01-25 15:12:08,5.000,0.000,0.000,2021,2
1,5.000,2021-02-04 21:05:32,2021-02-04 20:43:40,1.000,0.000,0.000,2021,2
2,4.500,2021-02-04 20:34:03,2019-07-04 23:23:27,1.000,0.000,0.000,2021,2
3,5.000,2021-02-04 16:56:28,2021-02-04 14:41:29,10.000,0.000,0.000,2021,2
4,4.000,2021-02-04 15:00:24,2020-10-13 03:10:07,10.000,0.000,0.000,2021,2


In [92]:
#yıl farkını bulma
dff['year_diff'] = date.today().year - dff['Timestamp'].dt.year

In [94]:
dff.head()

,Rating,Timestamp,Enrolled,Progress,Questions Asked,Questions Answered,year,month,year_diff
0,5.000,2021-02-05 07:45:55,2021-01-25 15:12:08,5.000,0.000,0.000,2021,2,4
1,5.000,2021-02-04 21:05:32,2021-02-04 20:43:40,1.000,0.000,0.000,2021,2,4
2,4.500,2021-02-04 20:34:03,2019-07-04 23:23:27,1.000,0.000,0.000,2021,2,4
3,5.000,2021-02-04 16:56:28,2021-02-04 14:41:29,10.000,0.000,0.000,2021,2,4
4,4.000,2021-02-04 15:00:24,2020-10-13 03:10:07,10.000,0.000,0.000,2021,2,4


In [96]:
#iki tarih arasındaki ay farkı
dff['month_diff'] = (date.today().year - dff['Timestamp'].dt.year)*12 + date.today().month - dff['Timestamp'].dt.month

In [98]:
#Gün ismini belirleme
dff['day_name'] = dff['Timestamp'].dt.day_name()

In [100]:
dff.head()

,Rating,Timestamp,Enrolled,Progress,Questions Asked,Questions Answered,year,month,year_diff,month_diff,day_name
0,5.000,2021-02-05 07:45:55,2021-01-25 15:12:08,5.000,0.000,0.000,2021,2,4,54,Friday
1,5.000,2021-02-04 21:05:32,2021-02-04 20:43:40,1.000,0.000,0.000,2021,2,4,54,Thursday
2,4.500,2021-02-04 20:34:03,2019-07-04 23:23:27,1.000,0.000,0.000,2021,2,4,54,Thursday
3,5.000,2021-02-04 16:56:28,2021-02-04 14:41:29,10.000,0.000,0.000,2021,2,4,54,Thursday
4,4.000,2021-02-04 15:00:24,2020-10-13 03:10:07,10.000,0.000,0.000,2021,2,4,54,Thursday


<span style="color: blue; font-weight: bold;">FEATURE INTERACTIONS(ÖZELLİK ETKİLEŞİMLERİ)</span>

In [104]:
df = load()
df.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.000,1,0,A/5 21171,7.250,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.000,1,0,PC 17599,71.283,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.000,0,0,STON/O2. 3101282,7.925,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.000,1,0,113803,53.100,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.000,0,0,373450,8.050,NaN,S


* Not: Değişkenlerin etkileşimi teorik olarak anlamlı bir bilgiye sahip olmalı

In [107]:
df['NEW_AGE_PCLASS'] = df['Age'] * df['Pclass']

In [ ]:
df['FAMILY_SIZE'] = df['SibSp']+df['Parch'] + 1#1 kişinin kendisi

In [ ]:
#NEW_SEX_CAT yeni bir sutun oluşturduk. Ve yaş aralağına göre gruplandırdık.
df.loc[(df['SEX'] == 'male') & (df['Age'] <= 21), 'NEW_SEX_CAT'] = 'youngmale'

df.loc[(df['SEX'] == 'male') & (df['Age'] > 21) & (df['Age'] < 50), 'NEW_SEX_CAT'] = 'maturemale'

df.loc[(df['SEX'] == 'male') & (df['Age'] >= 50), 'NEW_SEX_CAT'] = 'seniormale'

df.loc[(df['SEX'] == 'female') & (df['Age'] <= 21), 'NEW_SEX_CAT'] = 'youngfemale'

df.loc[(df['SEX'] == 'female') & (df['Age'] > 21) & (df['Age'] < 50), 'NEW_SEX_CAT'] = 'maturefemale'

df.loc[(df['SEX'] == 'female') & (df['Age'] >= 50), 'NEW_SEX_CAT'] = 'seniorfemale'